In [201]:
import pandas as pd
import html5lib
import requests
from bs4 import BeautifulSoup
import time
import urllib.request  # we are going to need to generate a Request object
from tqdm import tqdm
from dateutil.parser import parse
import re
from tqdm import tqdm

In [202]:
def transfermarkt_call(url):
    # here we define the headers for the request
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.13; rv:63.0) Gecko/20100101 Firefox/63.0'}

    # this request object will integrate your URL and the headers defined above
    req = urllib.request.Request(url=url, headers=headers)

    # calling urlopen this way will automatically handle closing the request
    with urllib.request.urlopen(req) as response:
        page_html = response.read()
    return page_html

# Gaeubschwarz

In [203]:
base_url = "https://www.gaeubschwarz.ch/category/matchbericht/spielerbewertungen/"
article_urls = [base_url]
for i in range(2,14):
    article_urls.append(base_url + "page/"+str(i)+"/")

In [204]:
game_articles = []
for article_url in article_urls:
    html = transfermarkt_call(article_url)
    soup = BeautifulSoup(html, 'html.parser')
    articles_obj = soup.find_all("article")

    for article in articles_obj:
        game_articles.append(article.a['href'])


In [243]:
player_df = pd.read_csv('../data/player_static.csv')

player_df = player_df.loc[player_df.team == "BSC Young Boys"]

player_lastname = player_df['name'].apply(lambda x: " ".join(x.split(" ")[1:]).lower())
player_df['player_lastname'] = list(player_lastname)

'David von Ballmoos'

In [264]:
data = {
    'name': [],
    'note': [],
    'date': []
}

for game_article in tqdm(game_articles):
    html = transfermarkt_call(game_article)
    soup = BeautifulSoup(html, 'html.parser')
    event_date = parse(game_article.replace("https://www.gaeubschwarz.ch/", "")[:10])
    
    noten = soup.find_all("strong")
    if len(noten)>0:
        noten_iter = iter(noten)
        next(noten_iter)
        for note in noten_iter:
            #note = ''.join(e for e in note.text if e.isalnum())
            splitted = note.text.split(" ")
            splitted = list(filter(None, splitted))
            splitted = [e.replace("/", "").replace(" ", "").replace(":", "").replace(",",".") for e in splitted]
            splitted = list(filter(None, splitted))

            #splitted = [e for e in splitted if float(e).isalnum()]
            
            if len(splitted) >= 2:
                if not splitted[0][0].isnumeric() and splitted[-1][0].isnumeric():
                    if not splitted[1][0].isnumeric():
                        splitted = [splitted[0] + " " + splitted[1]] + splitted[2:]
                    if not splitted[1][0].isnumeric():
                        splitted = [splitted[0] + " " + splitted[1]] + splitted[2:]
                    
                    splitted[1] = splitted[1].replace("/", "").replace(" ", "").replace(":", "")
                    
                    if splitted[0] != "Note" and "(" not in splitted[0]:
                        if len(player_df.loc[player_df['player_lastname']==splitted[0].lower()]) > 0:
                            data['name'].append(player_df.loc[player_df['player_lastname']==splitted[0].lower()]['name'].iloc[0])
                            data['note'].append(splitted[1])
                            data['date'].append(event_date)
                
    else:
        print(event_date)
            #print(splitted)
    
                # items is ("foo", "21")














  0%|          | 0/122 [00:00<?, ?it/s]











  1%|          | 1/122 [00:00<01:08,  1.78it/s]











  2%|▏         | 2/122 [00:01<01:06,  1.82it/s]











  2%|▏         | 3/122 [00:01<01:06,  1.79it/s]











  3%|▎         | 4/122 [00:02<01:05,  1.81it/s]











  4%|▍         | 5/122 [00:02<01:07,  1.75it/s]











  5%|▍         | 6/122 [00:03<01:04,  1.79it/s]











  6%|▌         | 7/122 [00:03<01:03,  1.81it/s]











  7%|▋         | 8/122 [00:04<01:02,  1.82it/s]











  7%|▋         | 9/122 [00:04<01:01,  1.84it/s]











  8%|▊         | 10/122 [00:05<01:00,  1.84it/s]











  9%|▉         | 11/122 [00:06<00:59,  1.86it/s]











 10%|▉         | 12/122 [00:06<00:59,  1.86it/s]











 11%|█         | 13/122 [00:07<00:58,  1.87it/s]











 11%|█▏        | 14/122 [00:07<00:57,  1.88it/s]











 12%|█▏        | 15/122 [00:08<00:56,  1.91it/s]











 13%|█▎        | 16/122 [00:08<00:56,  1.89it/

In [261]:
df = pd.DataFrame(data)


In [262]:
df

,name,note,date
0,David von Ballmoos,5,2019-11-08
1,Jordan Lotomba,3.5,2019-11-08
2,Frederik Sörensen,5.5,2019-11-08
3,Cédric Zesiger,5.5,2019-11-08
4,Ulisses Garcia,4,2019-11-08
...,...,...,...
278,Guillaume Hoarau,5,2014-10-20
279,Marco Wölfli,5,2013-11-25
280,Marco Wölfli,4.5,2013-08-12
281,Marco Wölfli,4.5,2013-07-21


In [263]:
df.to_csv('../data/player_noten.csv', index=False)

In [226]:
len(splitted) > 2

True

In [170]:
if not splitted[1][0].isnumeric():
    splitted = [splitted[0] + " " + splitted[1]] + splitted[2:]

In [221]:
"5,5"[0].isnumeric()

True

In [176]:
df = pd.DataFrame(data)
len(df)

114

In [80]:
player_df = pd.read_csv('../data/player_static.csv')

player_df = player_df.loc[player_df.team == "BSC Young Boys"]

player_lastname = player_df['name'].apply(lambda x: " ".join(x.split(" ")[1:]))
player_lastname = list(player_lastname)

['Von Ballmoos', '5']

In [106]:
res = re.findall(r'(\w+?)(\d+)', noten[3].text)
res

[]

In [79]:
noten = soup.find_all("strong", text="Noten")
noten

[]

[<strong>Die Noten</strong>,
 <strong>Von Ballmoos 5/</strong>,
 <strong>Lotomba 3,5/ </strong>,
 <strong>Sörensen 5,5/ </strong>,
 <strong>Zesiger 5,5/</strong>,
 <strong>Garcia 4 / </strong>,
 <strong>Lustenberger 4,5/</strong>,
 <strong>Aebischer 5/</strong>,
 <strong>Fassnacht 4,5 /</strong>,
 <strong>Ngamaleu 4/</strong>,
 <strong>Nsame 4,5/</strong>,
 <strong>Assalé 4,5/ </strong>,
 <strong>Spielmann 5/ </strong>,
 <strong>Janko/Mambimbi -/</strong>,
 <strong>Wen wünschst du dir als Gegner im Viertelfinal?</strong>]

In [45]:
columns = soup.findAll('p')
columns

[<p class="main-title bhf-hidden" itemprop="headline"><a href="https://www.gaeubschwarz.ch" rel="home" title="gaeubschwarz.ch">gaeubschwarz.ch</a></p>,
 <p>Soll man sich jetzt über diesen Punkt in Rotterdam freuen
 oder der verpassten Riesenchance von Spielmann nachtrauern? Wahrscheinlich
 können sich viele auch eine Nacht später noch nicht entscheiden.</p>,
 <p>Aber von Anfang an. YB darf gegen Feyenoord wieder auf
 Captain Lustenberger zurückgreifen. Dieser wirkt zwar nicht ganz fit, steht
 aber bis zum Schlusspfiff auf dem Feld und bringt allein durch seine Präsenz
 eine gewisse Stabilität ins YB-Spiel.</p>,
 <p>Die Holländer sind  zuhause
 von Beginn weg gefährlicher als noch im Wankdorf. Vor allem Neo-Captain
 Berghuis ist kaum zu stoppen und ist an jeder gefährlichen Aktion der
 Hausherren beteiligt. Doch wie bereits in Bern zeigt sich auch gestern: In der
 Defensive ist Rotterdam verwundbar.</p>,
 <p>Die erste dicke Chance gehört trotzdem den Hausherren. Sörensen
 kann den Absch